# EfficientNet-B0

## Setup

### Packages and Libraries

In [1]:
# Basic imports
import os
import time
import sys
from IPython.display import clear_output
import random
import numpy as np

import pandas as pd
import math


# Tensorflow and Keras
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import LearningRateScheduler

# Image processing
from PIL import Image
import cv2
from skimage import exposure
from scipy.ndimage import gaussian_filter

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

import torch

2024-04-14 10:26:10.143527: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 10:26:10.298978: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-14 10:26:11.085671: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-04-14 10:26:11.085813: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or 

### Configurations

In [2]:
# Set seeds to ensure reproducibility
np.random.seed(42)
random.seed(42)
tf.random.set_seed(42)

torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)  # For multi-GPU setups

In [3]:
num_classes = 5  # 5 for Ddr, 4 for others

In [4]:
# Set paths
base_data_path = "../Tina Codes/"  # <-- Add path to the numpy files here
output_path = "output/"  # <-- Add path to the output folder here

In [5]:
# Augmentation config

# decide whether to apply augmentation or not
apply_augmentation = False

# decide whether to apply brightness augmentation or not
aug_br_flag = 1  # 1 = Augment Brightness, anything else = No Brightness Augmentation

In [6]:
# Configure GPU memory growth
gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        # Set memory growth before initializing GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(
            "Num GPUs Available: Physical GPUs=",
            len(gpus),
            " | Logical GPUs=",
            len(logical_gpus),
        )
    except RuntimeError as e:
        # Handle potential errors here
        print(e)

Num GPUs Available: Physical GPUs= 1  | Logical GPUs= 1


2024-04-14 10:26:14.360616: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 10:26:14.659101: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 14791 MB memory:  -> device: 0, name: Tesla V100-PCIE-16GB, pci bus id: 0001:00:00.0, compute capability: 7.0


## Functions

In [7]:
# To display a progress bar
def update_progress(progress):
    # progress is a float between 0 and 1

    bar_length = 20
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait=True)
    text = "Progress: [{0}] {1:.1f}%".format(
        "#" * block + "-" * (bar_length - block), progress * 100
    )
    print(text)

In [8]:
## Augemntation Helper Fucntions
## image is an np array of the image
def rotate_image(image, angle):
    height, width = image.shape[:2]
    rotation_matrix = cv2.getRotationMatrix2D((width / 2, height / 2), angle, 1)
    rotated_image = cv2.warpAffine(image, rotation_matrix, (width, height))
    return rotated_image


def flip_image(image):
    flip_variations = [
        (False, False),  # No flip
        (True, False),  # Horizontal flip
        (False, True),  # Vertical flip
        (True, True),  # Both flips
    ]

    random.shuffle(flip_variations)
    augmented_images = []

    for flip_horizontal, flip_vertical in flip_variations[:4]:
        if flip_horizontal and flip_vertical:
            augmented_images.append(cv2.flip(image, -1))  # horizontal and vertical
        elif flip_horizontal:
            augmented_images.append(cv2.flip(image, 1))  # horizontal
        elif flip_vertical:
            augmented_images.append(cv2.flip(image, 0))  # vertical
        else:
            augmented_images.append(image)  # No flip

    return augmented_images


def shear_image(image, shear_range):
    height, width = image.shape[:2]
    shear_value = random.uniform(-shear_range, shear_range)

    if shear_value < 0:
        shear_matrix = np.array([[1, -shear_value, 0], [0, 1, 0]])
    else:
        shear_matrix = np.array([[1, shear_value, 0], [0, 1, 0]])

    sheared_image = cv2.warpAffine(image, shear_matrix, (width, height))
    return sheared_image


def translate_image(image, translate_range):
    height, width = image.shape[:2]

    max_shift_x = int(width * 0.1)
    max_shift_y = int(height * 0.1)

    translate_x = random.randint(-max_shift_x, max_shift_x)
    translate_y = random.randint(-max_shift_y, max_shift_y)

    translation_matrix = np.array(
        [[1, 0, translate_x], [0, 1, translate_y]], dtype=np.float32
    )

    translated_image = cv2.warpAffine(image, translation_matrix, (width, height))

    return translated_image


def adjust_brightness(image, brightness_range):
    brightness_factor = 1.0 + random.uniform(-brightness_range, brightness_range)
    adjusted_image = np.clip(image * brightness_factor, 0.25, 255).astype(np.uint8)
    return adjusted_image

In [9]:
## image parameter is an np array of an image
## flag 1: includes brightness, 0 or any other value exludes it


def augmented_fn2(image, flag):
    augmented_images = []

    for _ in range(4):
        angle = random.uniform(-35, 35)
        image_r = rotate_image(image, angle)
        augmented_images.append(image_r)

    image_f = flip_image(image)
    augmented_images.extend(image_f)

    for _ in range(4):
        image_s = shear_image(image, shear_range=0.15)
        augmented_images.append(image_s)

    for _ in range(4):
        image_t = translate_image(image, translate_range=0.1)
        augmented_images.append(image_t)

    if flag == 1:
        for _ in range(4):
            image_b = adjust_brightness(image, brightness_range=0.25)
            augmented_images.append(image_b)

    return np.array(augmented_images)

## Data Preparation

In [10]:
# Load the data
X_train = np.load(base_data_path + "X_train.npy")
y_train = np.load(base_data_path + "y_train.npy")
X_test = np.load(base_data_path + "X_test.npy")
y_test = np.load(base_data_path + "y_test.npy")

In [11]:
# Augmentation
if apply_augmentation:
    # temporarily store the augmented data
    X_train_aug = []
    y_train_aug = []

    for i in range(len(X_train)):
        X_train_aug.extend(augmented_fn2(X_train[i], aug_br_flag))
        y_train_aug.extend([y_train[i]] * 20)

    X_train = np.array(X_train_aug)
    y_train = np.array(y_train_aug)

In [12]:
# data validation

assert X_train.shape[0] == y_train.shape[0]
assert X_test.shape[0] == y_test.shape[0]
assert X_train.shape[1:] == X_test.shape[1:]

# Dimension check, for EfficientNetB0, the input shape is (224, 224, 3)
assert X_train.shape[1:] == (224, 224, 3)

print("Data loaded and augmented successfully, the data shapes are:")
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)

Data loaded and augmented successfully, the data shapes are:
X_train shape: (10019, 224, 224, 3)
y_train shape: (10019,)
X_test shape: (2503, 224, 224, 3)
y_test shape: (2503,)


In [13]:
# splitting train data into train and validation
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, test_size=0.2, random_state=42
)

In [14]:
# converting the data to tensor
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

## Model

In [15]:
# setting base model
base_model = tf.keras.applications.EfficientNetB0(
    include_top=False, weights="imagenet", input_shape=(224, 224, 3)
)

In [16]:
# Freeze the base model
base_model.trainable = False

In [17]:
# creating a new model on top of the base model
model = Sequential(
    [
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax"),
    ]
)

### Model Parameters

In [18]:
# Model parameters
batch_size = 32
epochs = 100

# learning rate scheduler
base_learning_rate = 1e-1
max_learning_rate = 1e-2


# creating a learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 15:
        return lr
    elif epoch < 24:
        return lr - 0.01
    elif epoch < 74:
        return lr
    else:
        return lr * tf.math.exp(-0.1)


# defining the optimizer
optimizer = SGD(learning_rate=base_learning_rate)

In [19]:
# compiling the model
model.compile(
    optimizer=optimizer,
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],
)
model.summary(show_trainable=True)

Model: "sequential"
____________________________________________________________________________
 Layer (type)                Output Shape              Param #   Trainable  
 efficientnetb0 (Functional)  (None, 7, 7, 1280)       4049571   N          
                                                                            
 global_average_pooling2d (G  (None, 1280)             0         Y          
 lobalAveragePooling2D)                                                     
                                                                            
 dense (Dense)               (None, 512)               655872    Y          
                                                                            
 dropout (Dropout)           (None, 512)               0         Y          
                                                                            
 dense_1 (Dense)             (None, 5)                 2565      Y          
                                                        

In [20]:
# Convert array-like data to tf.data.Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train_tensor, y_train_tensor))

# Shuffle and batch the dataset
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(batch_size)

# Convert array-like data to tf.data.Dataset
val_dataset = tf.data.Dataset.from_tensor_slices((X_val_tensor, y_val_tensor)).batch(
    batch_size
)

# converting test data to tf.data.Dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test_tensor, y_test_tensor)).batch(
    batch_size
)

2024-04-14 10:26:20.591518: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4825927680 exceeds 10% of free system memory.


In [21]:
# callbacks
early_stopping = EarlyStopping(
    restore_best_weights=True,  # monitor="val_loss", patience=10,
)

# training the model
history = model.fit(
    # X_train_tensor,
    # y_train_tensor,
    train_dataset,
    epochs=epochs,
    validation_data=val_dataset,
    # batch_size=batch_size,
    callbacks=[LearningRateScheduler(lr_scheduler)],
)

2024-04-14 10:26:23.867935: W tensorflow/core/framework/cpu_allocator_impl.cc:82] Allocation of 4825927680 exceeds 10% of free system memory.


Epoch 1/100


2024-04-14 10:26:33.326168: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8902
2024-04-14 10:26:33.435980: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


251/251 [==============================] - 18s 43ms/step - loss: 0.9681 - accuracy: 0.6153 - val_loss: 0.9161 - val_accuracy: 0.6502 - lr: 0.1000
Epoch 2/100
251/251 [==============================] - 8s 32ms/step - loss: 0.8590 - accuracy: 0.6629 - val_loss: 0.8457 - val_accuracy: 0.6911 - lr: 0.1000
Epoch 3/100
251/251 [==============================] - 8s 32ms/step - loss: 0.8327 - accuracy: 0.6731 - val_loss: 0.8587 - val_accuracy: 0.6667 - lr: 0.1000
Epoch 4/100
251/251 [==============================] - 8s 32ms/step - loss: 0.8064 - accuracy: 0.6868 - val_loss: 0.7861 - val_accuracy: 0.6961 - lr: 0.1000
Epoch 5/100
251/251 [==============================] - 8s 32ms/step - loss: 0.7941 - accuracy: 0.6898 - val_loss: 0.8214 - val_accuracy: 0.7091 - lr: 0.1000
Epoch 6/100
251/251 [==============================] - 8s 32ms/step - loss: 0.7737 - accuracy: 0.7034 - val_loss: 0.7920 - val_accuracy: 0.7016 - lr: 0.1000
Epoch 7/100
251/251 [==============================] - 8s 32ms/step -

In [22]:
# evaluating the model
test_loss, test_acc = model.evaluate(test_dataset)
print(f"Test accuracy:{test_acc:.2%}")

79/79 [==============================] - 3s 28ms/step - loss: 0.6526 - accuracy: 0.7579
Test accuracy:75.79%
